Reproducible steps to create a corpus from [FinnHub](https://finnhub.io/).

# Pseudocode

Below is a list of the steps we take.
Keep in mind that these steps are a 10 thousand foot view.
The implementation will be commented to a more detailed level.

1. Get the tickers from the [SEC](https://www.sec.gov/file/company-tickers)
2. Using the retrieved data, get the tickers for every publicly traded stock in the U.S. market.

In [ ]:
import requests
import pandas as pd
import json
import csv
import os

from pathlib import Path
from tqdm.notebook import tqdm
from dotenv import load_dotenv, dotenv_values 

In [ ]:
tickers_url = 'https://www.sec.gov/files/company_tickers.json'
user_agent = 'FinnHub-Data-Ingestion'
limit = 20

data_folder = Path('./data/')
tickers_file = data_folder.joinpath('./tickers.csv')
raw_folder = data_folder.joinpath('./raw')
corpus_folder = data_folder.joinpath('./corpus')

# Step 1

1. Get the list of tickers from the SEC
2. Convert the tickers into an array, then sort it.
3. Save the tickers to a CSV

In [ ]:
def get_tickers(tickers_file: Path, tickers_url: str, user_agent: str, ) -> pd.DataFrame:
    if not tickers_file.exists():
        tickers = None
        with requests.Session() as session:
            session.headers['User-Agent'] = user_agent
            with session.get(tickers_url) as result:
                if result.status_code == 200:
                    t1 = json.loads(result.text)
                    t2 = [x for x in t1.values()]
                    t3 = sorted(t2, key = lambda tup: tup['ticker'])
                    tickers = [(x['cik_str'], x['ticker'], x['title']) for x in t3]
        if tickers is not None:
            df = pd.DataFrame(tickers, columns = ['CIK', 'Ticker', 'Name'])
            if not tickers_file.parent.exists():
                tickers_file.parent.mkdir(parents = True)
            df.to_csv(tickers_file, index = False)
        else:
            raise RuntimeError('Error retrieving tickers')          
    return pd.read_csv(tickers_file) #type: ignore

tickers_df = get_tickers(tickers_file, tickers_url, user_agent)

# Step 2

1. Iterate through each ticker in the `tickers.csv` file and download all available trading data.
2. Save data of each ticker to JSON files.

In [ ]:
load_dotenv()

base_url = f'https://finnhub.io/api/v1/stock/congressional-trading?symbol='

headers = {
        'Accept': '/',
        'User-Agent': 'Thunder Client (https://www.thunderclient.com)',
        'X-FinnHub-Token': os.getenv("API_KEY"), # Replace with your actual API key
                }

tickers = tickers_df['Ticker']

In [ ]:
for ticker in tqdm(tickers):
    url = base_url + str(ticker)
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        if 'transactionDate' in data:
            filename = f'data/raw/{ticker}.json'
            with open(filename, 'w') as jsonfile:
                jsonfile.write(response.text)
            print(f'Response for {ticker} saved to {filename}')
        else:
            print(f'No trade data available for {ticker}. Skipping.')
    else:
        print(f'Failed to retrieve data for {ticker}. Status code: {response.status_code}')

print("Data download completed.")